## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pangnirtung,CA,66.1451,-65.7125,-15.21,100,100,5.73,overcast clouds
1,1,Pakxan,LA,18.3778,103.6601,68.72,89,88,3.44,overcast clouds
2,2,Ponta Do Sol,PT,32.6667,-17.1000,64.06,58,30,5.12,scattered clouds
3,3,New Norfolk,AU,-42.7826,147.0587,65.70,56,99,4.72,overcast clouds
4,4,Kyren,RU,51.6828,102.1408,-14.55,89,88,1.36,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Pakxan,LA,18.3778,103.6601,68.72,89,88,3.44,overcast clouds
2,2,Ponta Do Sol,PT,32.6667,-17.1000,64.06,58,30,5.12,scattered clouds
3,3,New Norfolk,AU,-42.7826,147.0587,65.70,56,99,4.72,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.32,69,1,18.16,light rain
7,7,Cidreira,BR,-30.1811,-50.2056,75.29,79,13,20.38,few clouds
9,9,Puerto Quijarro,BO,-17.7833,-57.7667,85.21,67,75,2.24,broken clouds
10,10,Saint-Joseph,RE,-21.3667,55.6167,77.36,79,18,9.62,few clouds
11,11,Pasighat,IN,28.0667,95.3333,52.00,73,0,4.03,clear sky
15,15,Mataura,NZ,-46.1927,168.8643,50.49,84,98,4.65,overcast clouds
16,16,Cape Town,ZA,-33.9258,18.4232,70.09,79,20,16.11,few clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                412
City                   412
Country                408
Lat                    412
Lng                    412
Max Temp               412
Humidity               412
Cloudiness             412
Wind Speed             412
Current Description    412
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
